#### Installing the required libraries

In [ ]:
!pip install torchinfo

#### Loading in the required libraries

In [ ]:
import torch
import torch.nn as nn
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torchvision.models import resnet50
from torchvision.models import ResNet50_Weights
from torchvision.datasets import MNIST, USPS, SVHN
from torchinfo import summary
from tqdm import tqdm
import torch.optim as optim
from sklearn.manifold import TSNE
import numpy as np
import random
from torch.autograd import Function
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/ATML_PA5'
# Setting the path to this folder
os.chdir(path)
# Checking the current working directory
print(os.getcwd())

/content/drive/MyDrive/ATML_PA5


#### Setting the device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("The device is: ", device)

The device is:  cuda


## Loading in the Datasets

In [ ]:
# Function to create a subset
def create_subset(dataset, subset_size, seed=42):
    """
    Creates a random subset of the dataset.

    Args:
        dataset (Dataset): The original dataset.
        subset_size (int): The number of samples in the subset.
        seed (int): Random seed for reproducibility.

    Returns:
        Subset: A PyTorch Subset object.
    """
    random.seed(seed)
    indices = random.sample(range(len(dataset)), subset_size)
    return Subset(dataset, indices)

#### Office-31

In [ ]:
data_path = 'OFFICE31'
amazon_path = os.path.join(data_path, 'amazon')
webcam_path = os.path.join(data_path, 'webcam')
dslr_path = os.path.join(data_path, 'dslr')

print('amazon_path:', amazon_path)
print('webcam_path:', webcam_path)
print('dslr_path:', dslr_path)

def load_data(root_path, domain, batch_size, phase):
    transform_dict = {
        'src': transforms.Compose(
        [transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
         ]),
        'tar': transforms.Compose(
        [transforms.Resize(224),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
         ])}
    data = datasets.ImageFolder(root=os.path.join(root_path, domain), transform=transform_dict[phase])
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=phase=='src', drop_last=phase=='tar', num_workers=4)
    return data_loader

amazon_loader = load_data(data_path, 'amazon', 64, 'src')
webcam_loader = load_data(data_path, 'webcam', 64, 'tar')
dslr_loader = load_data(data_path, 'dslr', 64, 'tar')

# Checking the size of these data loaders
print('amazon_loader size:', len(amazon_loader))
print('webcam_loader size:', len(webcam_loader))
print('dslr_loader size:', len(dslr_loader))

# # Check the size of the first batch
# amazon_data = next(iter(amazon_loader))
# webcam_data = next(iter(webcam_loader))
# dslr_data = next(iter(dslr_loader))

# print('amazon_data size:', amazon_data[0].size())
# print('webcam_data size:', webcam_data[0].size())
# print('dslr_data size:', dslr_data[0].size())


amazon_path: OFFICE31/amazon
webcam_path: OFFICE31/webcam
dslr_path: OFFICE31/dslr


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


amazon_loader size: 45
webcam_loader size: 12
dslr_loader size: 7


#### Digits Datasets

In [ ]:
# Define transformations for the datasets
transform_mnist_usps = transforms.Compose([
    transforms.Resize((224, 224)),                  # Resize to 224x224
    transforms.Grayscale(num_output_channels=3),  # Ensure grayscale images (for USPS/MNIST) and convert to 3 channels
    transforms.ToTensor(),                        # Convert to Tensor
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
    # transforms.Normalize((0.5,), (0.5,))          # Normalize to [-1, 1]
])

transform_svhn = transforms.Compose([
    transforms.Resize((224, 224)),                  # Resize to 224x224
    transforms.ToTensor(),                        # Convert to Tensor
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
    # transforms.Normalize((0.5,), (0.5,))          # Normalize to [-1, 1]
])

batch_size = 8

# Loading in MNIST dataset
mnist_train = MNIST(root='./data', train=True, download=True, transform=transform_mnist_usps)
mnist_test = MNIST(root='./data', train=False, download=True, transform=transform_mnist_usps)

# MNIST dataloaders
mnist_train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
mnist_test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

# Loading in the USPS dataset
usps_train = USPS(root='./data', train=True, download=True, transform=transform_mnist_usps)
usps_test = USPS(root='./data', train=False, download=True, transform=transform_mnist_usps)

# USPS dataloaders
usps_train_loader = DataLoader(usps_train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
usps_test_loader = DataLoader(usps_test, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

# Loading in the SVHN dataset
svhn_train = SVHN(root='./data', split='train', download=True, transform=transform_svhn)
svhn_test = SVHN(root='./data', split='test', download=True, transform=transform_svhn)

# SVHN dataloaders
svhn_train_loader = DataLoader(svhn_train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
svhn_test_loader = DataLoader(svhn_test, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

# Print dataset sizes for verification
print(f"MNIST Train Size: {len(mnist_train)}")
print(f"MNIST Test Size: {len(mnist_test)}")
print(f"USPS Train Size: {len(usps_train)}")
print(f"USPS Test Size: {len(usps_test)}")
print(f"SVHN Train Size: {len(svhn_train)}")
print(f"SVHN Test Size: {len(svhn_test)}")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
MNIST Train Size: 60000
MNIST Test Size: 10000
USPS Train Size: 7291
USPS Test Size: 2007
SVHN Train Size: 73257
SVHN Test Size: 26032


In [ ]:
# Create subsets of the train and test datasets
mnist_train_subset = create_subset(mnist_train, 10000)
mnist_test_subset = create_subset(mnist_test, 5000)
usps_train_subset = create_subset(usps_train, 5000)
usps_test_subset = create_subset(usps_test, 1000)
svhn_train_subset = create_subset(svhn_train, 10000)
svhn_test_subset = create_subset(svhn_test, 5000)

# Creating the subset dataloaders
mnist_train_subset_loader = DataLoader(mnist_train_subset, batch_size=batch_size, shuffle=True)
mnist_test_subset_loader = DataLoader(mnist_test_subset, batch_size=batch_size, shuffle=False)
usps_train_subset_loader = DataLoader(usps_train_subset, batch_size=batch_size, shuffle=True)
usps_test_subset_loader = DataLoader(usps_test_subset, batch_size=batch_size, shuffle=False)
svhn_train_subset_loader = DataLoader(svhn_train_subset, batch_size=batch_size, shuffle=True)
svhn_test_subset_loader = DataLoader(svhn_test_subset, batch_size=batch_size, shuffle=False)


# Check the size of the subsets
print(f"MNIST Train Subset Size: {len(mnist_train_subset)}")
print(f"MNIST Test Subset Size: {len(mnist_test_subset)}")
print(f"USPS Train Subset Size: {len(usps_train_subset)}")
print(f"USPS Test Subset Size: {len(usps_test_subset)}")
print(f"SVHN Train Subset Size: {len(svhn_train_subset)}")
print(f"SVHN Test Subset Size: {len(svhn_test_subset)}")


MNIST Train Subset Size: 10000
MNIST Test Subset Size: 5000
USPS Train Subset Size: 5000
USPS Test Subset Size: 1000
SVHN Train Subset Size: 10000
SVHN Test Subset Size: 5000


## DANN Model

#### Util functions

In [ ]:
class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

def optimizer_scheduler(optimizer, p):
    """
    Adjust the learning rate of optimizer
    :param optimizer: optimizer for updating parameters
    :param p: a variable for adjusting learning rate
    :return: optimizer
    """
    for param_group in optimizer.param_groups:
        param_group['lr'] = 0.01 / (1. + 10 * p) ** 0.75

    return optimizer

#### DANN Class

In [ ]:
# class DANN(nn.Module):
#     def __init__(self, num_classes, alpha=1.0):
#         super(DANN, self).__init__()
#         self.alpha = alpha
#         self.feature_extractor = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
#         num_features = self.feature_extractor.fc.in_features
#         self.feature_extractor.fc = nn.Identity()
#         self.class_classifier = nn.Sequential(
#             nn.Linear(num_features, 512),
#             nn.ReLU(),
#             nn.Linear(512, 256),
#             nn.ReLU(),
#             nn.Linear(256, num_classes)
#         )
#         self.domain_classifier = nn.Sequential(
#             nn.Linear(num_features, 512),
#             nn.ReLU(),
#             nn.Linear(512, 256),
#             nn.ReLU(),
#             nn.Linear(256, 1)
#         )

#     def forward(self, x, alpha=None):
#         if alpha is None:
#             alpha = self.alpha
#         features = self.feature_extractor(x)
#         reverse_features = ReverseLayerF.apply(features, alpha)
#         class_output = self.class_classifier(features)
#         domain_output = self.domain_classifier(reverse_features)
#         return class_output, domain_output


# # Create the model
# num_classes = 31
# model = DANN(num_classes, 1.0).to(device)
# summary(model, input_size=(64, 3, 224, 224))

class Extractor(nn.Module):
    def __init__(self):
        super(Extractor, self).__init__()
        self.feature_extractor = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        num_features = self.feature_extractor.fc.in_features
        self.feature_extractor.fc = nn.Identity()

    def forward(self, x):
        features = self.feature_extractor(x)
        return features

class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.class_classifier = nn.Sequential(
            nn.Linear(2048, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, num_classes)
        )

    def forward(self, x):
        class_output = self.class_classifier(x)
        return class_output

class DomainClassifier(nn.Module):
    def __init__(self):
        super(DomainClassifier, self).__init__()
        self.domain_classifier = nn.Sequential(
            nn.Linear(2048, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 2)
        )

    def forward(self, x, alpha):
        reversed_input = ReverseLayerF.apply(x, alpha)
        domain_output = self.domain_classifier(reversed_input)
        return domain_output

# Testing the flow of the model
extractor = Extractor().to(device)
classifier = Classifier(31).to(device)
domain_classifier = DomainClassifier().to(device)

# Testing the flow of the model
x = torch.randn(64, 3, 224, 224).to(device)
features = extractor(x)
print("Feature shape:", features.shape)

class_output = classifier(features)
print("Class output shape:", class_output.shape)

domain_output = domain_classifier(features, 1.0)
print("Domain output shape:", domain_output.shape)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 176MB/s]


Feature shape: torch.Size([64, 2048])
Class output shape: torch.Size([64, 31])
Domain output shape: torch.Size([64, 2])


#### Defining the training and testing functions

In [ ]:
# def train_dann_model(model, source_loader, target_loader, num_epochs, loss_class, loss_domain, optimizer, source_name, target_name, device):
#     model.train()
#     for epoch in range(num_epochs):
#         total_loss = 0.0
#         correct_class = 0
#         correct_domain = 0
#         total_samples = 0

#         source_iter = iter(source_loader)
#         target_iter = iter(target_loader)
#         num_batches = min(len(source_iter), len(target_iter))

#         with tqdm(total=num_batches, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
#             for _ in range(num_batches):
#                 source_data, source_labels = next(source_iter)
#                 target_data, _ = next(target_iter)

#                 source_data, source_labels = source_data.to(device), source_labels.to(device)
#                 target_data = target_data.to(device)

#                 optimizer.zero_grad()

#                 # Forward pass
#                 class_output, domain_output = model(source_data)
#                 _, target_domain_output = model(target_data)

#                 # Compute losses
#                 loss_s_label = loss_class(class_output, source_labels)
#                 loss_s_domain = loss_domain(domain_output, torch.zeros_like(domain_output))
#                 loss_t_domain = loss_domain(target_domain_output, torch.ones_like(target_domain_output))

#                 loss = loss_s_label + loss_s_domain + loss_t_domain
#                 total_loss += loss.item()

#                 # Backward pass and optimization
#                 loss.backward()
#                 optimizer.step()

#                 # Compute accuracies
#                 _, predicted_class = torch.max(class_output, 1)
#                 correct_class += (predicted_class == source_labels).sum().item()

#                 _, predicted_domain = torch.max(domain_output, 1)
#                 correct_domain += (predicted_domain == torch.zeros_like(domain_output)).sum().item()

#                 _, predicted_target_domain = torch.max(target_domain_output, 1)
#                 correct_domain += (predicted_target_domain == torch.ones_like(target_domain_output)).sum().item()

#                 total_samples += source_labels.size(0)

#                 pbar.set_postfix(loss=total_loss / total_samples, class_acc=correct_class / total_samples, domain_acc=correct_domain / (2 * total_samples))
#                 pbar.update(1)

#         print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/total_samples:.4f}, Class Accuracy: {correct_class/total_samples:.4f}, Domain Accuracy: {correct_domain/(2*total_samples):.4f}")

def tester(encoder, classifier, discriminator, source_test_loader, target_test_loader, training_mode, device):
    encoder.to(device)
    classifier.to(device)

    # Set models to eval mode
    encoder.eval()
    classifier.eval()

    if training_mode == 'DANN':
        discriminator.to(device)
        discriminator.eval()
        domain_correct = 0

    source_correct = 0
    target_correct = 0

    # Combine source and target test loaders with tqdm progress bar
    total_batches = min(len(source_test_loader), len(target_test_loader))
    with tqdm(total=total_batches, desc="Testing", unit="batch") as progress_bar:
        for batch_idx, (source_data, target_data) in enumerate(zip(source_test_loader, target_test_loader)):
            p = float(batch_idx) / total_batches
            alpha = 2. / (1. + np.exp(-10 * p)) - 1

            # Process source and target data
            source_image, source_label = source_data
            target_image, target_label = target_data

            source_image, source_label = source_image.to(device), source_label.to(device)
            target_image, target_label = target_image.to(device), target_label.to(device)

            # Compute source and target predictions
            source_pred = compute_output(encoder, classifier, source_image, alpha=None)
            target_pred = compute_output(encoder, classifier, target_image, alpha=None)

            # Update correct counts
            source_correct += source_pred.eq(source_label.data.view_as(source_pred)).sum().item()
            target_correct += target_pred.eq(target_label.data.view_as(target_pred)).sum().item()

            if training_mode == 'DANN':
                # Process combined images for domain classification
                combined_image = torch.cat((source_image, target_image), 0)
                domain_labels = torch.cat((torch.zeros(source_label.size(0), dtype=torch.long),
                                           torch.ones(target_label.size(0), dtype=torch.long)), 0).to(device)

                # Compute domain predictions
                domain_pred = compute_output(encoder, discriminator, combined_image, alpha=alpha)
                domain_correct += domain_pred.eq(domain_labels.data.view_as(domain_pred)).sum().item()

            # Update tqdm progress bar
            progress_bar.update(1)

    source_dataset_len = len(source_test_loader.dataset)
    target_dataset_len = len(target_test_loader.dataset)

    accuracies = {
        "Source": {
            "correct": source_correct,
            "total": source_dataset_len,
            "accuracy": calculate_accuracy(source_correct, source_dataset_len)
        },
        "Target": {
            "correct": target_correct,
            "total": target_dataset_len,
            "accuracy": calculate_accuracy(target_correct, target_dataset_len)
        }
    }

    if training_mode == 'DANN':
        accuracies["Domain"] = {
            "correct": domain_correct,
            "total": source_dataset_len + target_dataset_len,
            "accuracy": calculate_accuracy(domain_correct, source_dataset_len + target_dataset_len)
        }

    print_accuracy(training_mode, accuracies)

def compute_output(encoder, classifier, images, alpha=None):
    features = encoder(images)
    if isinstance(classifier, DomainClassifier):
        outputs = classifier(features, alpha)  # Domain classifier
    else:
        outputs = classifier(features)  # Category classifier
    preds = outputs.data.max(1, keepdim=True)[1]
    return preds


def calculate_accuracy(correct, total):
    return 100. * correct / total


def print_accuracy(training_mode, accuracies):
    print(f"Test Results on {training_mode}:")
    for key, value in accuracies.items():
        print(f"{key} Accuracy: {value['correct']}/{value['total']} ({value['accuracy']:.2f}%)")

def train_dann(encoder, classifier, discriminator, source_train_loader, target_train_loader, epochs, device, source_test_loader, target_test_loader):
    print("Training with the DANN adaptation method")

    classifier_criterion = nn.CrossEntropyLoss().to(device)
    discriminator_criterion = nn.CrossEntropyLoss().to(device)

    optimizer = optim.SGD(
        list(encoder.parameters()) +
        list(classifier.parameters()) +
        list(discriminator.parameters()),
        lr=0.01,
        momentum=0.9)

    for epoch in range(epochs):
        print(f"Epoch: {epoch + 1}/{epochs}")

        epoch_total_loss = 0.0
        epoch_class_loss = 0.0
        epoch_domain_loss = 0.0

        # Setting the models to train mode
        encoder.train()
        classifier.train()
        discriminator.train()

        start_steps = epoch * len(source_train_loader)
        total_steps = epochs * len(target_train_loader)

        # Use tqdm to track the progress of the batch loop
        progress_bar = tqdm(zip(source_train_loader, target_train_loader), total=min(len(source_train_loader), len(target_train_loader)), desc=f"Epoch {epoch + 1}/{epochs}", leave=False)

        for batch_idx, (source_data, target_data) in enumerate(progress_bar):
            source_image, source_label = source_data
            target_image, target_label = target_data

            p = float(batch_idx + start_steps) / total_steps
            alpha = 2. / (1. + np.exp(-10 * p)) - 1

            source_image, source_label = source_image.to(device), source_label.to(device)
            target_image, target_label = target_image.to(device), target_label.to(device)

            combined_image = torch.cat((source_image, target_image), 0)

            optimizer = optimizer_scheduler(optimizer=optimizer, p=p)
            optimizer.zero_grad()

            combined_feature = encoder(combined_image)
            source_feature = encoder(source_image)

            # 1. Classification loss
            class_pred = classifier(source_feature)
            class_loss = classifier_criterion(class_pred, source_label)

            # 2. Domain loss
            domain_pred = discriminator(combined_feature, alpha)

            domain_source_labels = torch.zeros(source_label.shape[0]).type(torch.LongTensor).to(device)
            domain_target_labels = torch.ones(target_label.shape[0]).type(torch.LongTensor).to(device)
            domain_combined_label = torch.cat((domain_source_labels, domain_target_labels), 0)
            domain_loss = discriminator_criterion(domain_pred, domain_combined_label)

            total_loss = class_loss + domain_loss
            total_loss.backward()
            optimizer.step()

            epoch_total_loss += total_loss.item()
            epoch_class_loss += class_loss.item()
            epoch_domain_loss += domain_loss.item()

            # Update tqdm bar with loss information
            progress_bar.set_postfix({
                'Total Loss': f'{total_loss.item():.4f}',
                'Class Loss': f'{class_loss.item():.4f}',
                'Domain Loss': f'{domain_loss.item():.4f}'
            })

            # End of epoch: Print average losses
            avg_total_loss = epoch_total_loss / len(source_train_loader)
            avg_class_loss = epoch_class_loss / len(source_train_loader)
            avg_domain_loss = epoch_domain_loss / len(source_train_loader)

            # print(f"Epoch {epoch + 1}/{epochs} Summary:")
            # print(f"  Average Total Loss: {avg_total_loss:.4f}")
            # print(f"  Average Classification Loss: {avg_class_loss:.4f}")
            # print(f"  Average Domain Loss: {avg_domain_loss:.4f}")

        # tester(encoder, classifier, discriminator, source_train_loader, target_train_loader, training_mode='DANN', device=device)
        tester(encoder, classifier, discriminator, source_test_loader, target_test_loader, training_mode='DANN', device=device)

        print(f"Epoch {epoch + 1}/{epochs} completed.")


In [ ]:
torch.cuda.empty_cache()

#### A -> W

In [ ]:
amazon_loader = load_data(data_path, 'amazon', 16, 'src')
webcam_loader = load_data(data_path, 'webcam', 16, 'tar')

print("Number of labels in Amazon dataset:", len(amazon_loader.dataset.classes))

# Create the model
encoder = Extractor().to(device)
classifier = Classifier(31).to(device)
discriminator = DomainClassifier().to(device)


# Train the model
# train_dann_model(model, amazon_loader, webcam_loader, 10, loss_class, loss_domain, optimizer, "amazon", "webcam", device)
# train_dann_model(model, amazon_loader, webcam_loader, 5, loss_class, loss_domain, optimizer, "amazon", "webcam", device)
train_dann(encoder, classifier, discriminator, amazon_loader, webcam_loader, 5, device)

Number of labels in Amazon dataset: 31
Training with the DANN adaptation method
Epoch: 1/5


Testing: 100%|██████████| 49/49 [01:04<00:00,  1.32s/batch]

Test Results on DANN:
Source Accuracy: 56/2837 (1.97%)
Target Accuracy: 60/795 (7.55%)
Domain Accuracy: 1343/3632 (36.98%)
Epoch 1/5 completed.
Epoch: 2/5



Testing: 100%|██████████| 49/49 [00:43<00:00,  1.13batch/s]

Test Results on DANN:
Source Accuracy: 69/2837 (2.43%)
Target Accuracy: 70/795 (8.81%)
Domain Accuracy: 1236/3632 (34.03%)
Epoch 2/5 completed.
Epoch: 3/5


...

Test Results on DANN:
Source Accuracy: 87/2837 (3.07%)
Target Accuracy: 84/795 (10.57%)
Domain Accuracy: 1273/3632 (35.05%)
Epoch 3/5 completed.
Epoch: 4/5



Testing: 100%|██████████| 49/49 [00:15<00:00,  3.13batch/s]

Test Results on DANN:
Source Accuracy: 83/2837 (2.93%)
Target Accuracy: 66/795 (8.30%)
Domain Accuracy: 932/3632 (25.66%)
Epoch 4/5 completed.
Epoch: 5/5



Testing: 100%|██████████| 49/49 [00:13<00:00,  3.66batch/s]

Test Results on DANN:
Source Accuracy: 70/2837 (2.47%)
Target Accuracy: 58/795 (7.30%)
Domain Accuracy: 824/3632 (22.69%)
Epoch 5/5 completed.


#### A -> D

In [ ]:
amazon_loader = load_data(data_path, 'amazon', 16, 'src')
dslr_loader = load_data(data_path, 'dslr', 16, 'tar')

print("Number of labels in Amazon dataset:", len(amazon_loader.dataset.classes))

# Create the model
encoder = Extractor().to(device)
classifier = Classifier(31).to(device)
discriminator = DomainClassifier().to(device)


# Train the model
train_dann(encoder, classifier, discriminator, amazon_loader, dslr_loader, 5, device)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Number of labels in Amazon dataset: 31
Training with the DANN adaptation method
Epoch: 1/5


Testing: 100%|██████████| 31/31 [00:12<00:00,  2.44batch/s]

Test Results on DANN:
Source Accuracy: 30/2837 (1.06%)
Target Accuracy: 38/508 (7.48%)
Domain Accuracy: 666/3345 (19.91%)
Epoch 1/5 completed.
Epoch: 2/5



Testing: 100%|██████████| 31/31 [00:11<00:00,  2.65batch/s]

...

Test Results on DANN:
Source Accuracy: 24/2837 (0.85%)
Target Accuracy: 44/508 (8.66%)
Domain Accuracy: 695/3345 (20.78%)
Epoch 3/5 completed.
Epoch: 4/5



Testing: 100%|██████████| 31/31 [00:11<00:00,  2.59batch/s]

Test Results on DANN:
Source Accuracy: 35/2837 (1.23%)
Target Accuracy: 42/508 (8.27%)
Domain Accuracy: 760/3345 (22.72%)
Epoch 4/5 completed.
Epoch: 5/5



Testing: 100%|██████████| 31/31 [00:15<00:00,  1.98batch/s]

Test Results on DANN:
Source Accuracy: 38/2837 (1.34%)
Target Accuracy: 50/508 (9.84%)
Domain Accuracy: 740/3345 (22.12%)
Epoch 5/5 completed.


#### W -> A

In [ ]:
webcam_loader = load_data(data_path, 'webcam', 16, 'src')
amazon_loader = load_data(data_path, 'amazon', 16, 'tar')

print("Number of labels in Amazon dataset:", len(amazon_loader.dataset.classes))

# Create the model
encoder = Extractor().to(device)
classifier = Classifier(31).to(device)
discriminator = DomainClassifier().to(device)


# Train the model
train_dann(encoder, classifier, discriminator, webcam_loader, amazon_loader, 5, device)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Number of labels in Amazon dataset: 31
Training with the DANN adaptation method
Epoch: 1/5


Testing: 100%|██████████| 50/50 [00:12<00:00,  4.14batch/s]

Test Results on DANN:
Source Accuracy: 129/795 (16.23%)
Target Accuracy: 85/2837 (3.00%)
Domain Accuracy: 1359/3632 (37.42%)
Epoch 1/5 completed.
Epoch: 2/5



Testing: 100%|██████████| 50/50 [00:13<00:00,  3.63batch/s]

...

Test Results on DANN:
Source Accuracy: 302/795 (37.99%)
Target Accuracy: 269/2837 (9.48%)
Domain Accuracy: 802/3632 (22.08%)
Epoch 3/5 completed.
Epoch: 4/5



Testing: 100%|██████████| 50/50 [00:13<00:00,  3.66batch/s]

Test Results on DANN:
Source Accuracy: 497/795 (62.52%)
Target Accuracy: 391/2837 (13.78%)
Domain Accuracy: 970/3632 (26.71%)
Epoch 4/5 completed.
Epoch: 5/5



Testing: 100%|██████████| 50/50 [00:13<00:00,  3.80batch/s]


Test Results on DANN:
Source Accuracy: 636/795 (80.00%)
Target Accuracy: 457/2837 (16.11%)
Domain Accuracy: 999/3632 (27.51%)
Epoch 5/5 completed.


#### W -> D

In [ ]:
webcam_loader = load_data(data_path, 'webcam', 16, 'src')
dslr_loader = load_data(data_path, 'dslr', 16, 'tar')

print("Number of labels in Amazon dataset:", len(amazon_loader.dataset.classes))

# Create the model
encoder = Extractor().to(device)
classifier = Classifier(31).to(device)
discriminator = DomainClassifier().to(device)


# Train the model
train_dann(encoder, classifier, discriminator, webcam_loader, dslr_loader, 5, device)

Number of labels in Amazon dataset: 31
Training with the DANN adaptation method
Epoch: 1/5


Testing: 100%|██████████| 31/31 [00:11<00:00,  2.61batch/s]

Test Results on DANN:
Source Accuracy: 41/795 (5.16%)
Target Accuracy: 24/508 (4.72%)
Domain Accuracy: 521/1303 (39.98%)
Epoch 1/5 completed.
Epoch: 2/5



Testing: 100%|██████████| 31/31 [00:16<00:00,  1.88batch/s]

Test Results on DANN:
Source Accuracy: 46/795 (5.79%)
Target Accuracy: 25/508 (4.92%)
Domain Accuracy: 677/1303 (51.96%)
Epoch 2/5 completed.
Epoch: 3/5


...

Test Results on DANN:
Source Accuracy: 49/795 (6.16%)
Target Accuracy: 26/508 (5.12%)
Domain Accuracy: 747/1303 (57.33%)
Epoch 3/5 completed.
Epoch: 4/5



Testing: 100%|██████████| 31/31 [00:16<00:00,  1.85batch/s]

Test Results on DANN:
Source Accuracy: 61/795 (7.67%)
Target Accuracy: 33/508 (6.50%)
Domain Accuracy: 741/1303 (56.87%)
Epoch 4/5 completed.
Epoch: 5/5



Testing: 100%|██████████| 31/31 [00:10<00:00,  2.86batch/s]

Test Results on DANN:
Source Accuracy: 67/795 (8.43%)
Target Accuracy: 35/508 (6.89%)
Domain Accuracy: 723/1303 (55.49%)
Epoch 5/5 completed.


#### D -> A

In [ ]:
dslr_loader = load_data(data_path, 'dslr', 16, 'src')
amazon_loader = load_data(data_path, 'amazon', 16, 'tar')

print("Number of labels in Amazon dataset:", len(amazon_loader.dataset.classes))

# Create the model
encoder = Extractor().to(device)
classifier = Classifier(31).to(device)
discriminator = DomainClassifier().to(device)


# Train the model
train_dann(encoder, classifier, discriminator, dslr_loader, amazon_loader, 5, device)

Number of labels in Amazon dataset: 31
Training with the DANN adaptation method
Epoch: 1/5


Testing: 100%|██████████| 32/32 [00:09<00:00,  3.31batch/s]

Test Results on DANN:
Source Accuracy: 37/508 (7.28%)
Target Accuracy: 0/2837 (0.00%)
Domain Accuracy: 940/3345 (28.10%)
Epoch 1/5 completed.
Epoch: 2/5



Testing: 100%|██████████| 32/32 [00:13<00:00,  2.34batch/s]

Test Results on DANN:
Source Accuracy: 102/508 (20.08%)
Target Accuracy: 72/2837 (2.54%)
Domain Accuracy: 895/3345 (26.76%)
Epoch 2/5 completed.
Epoch: 3/5


...

Test Results on DANN:
Source Accuracy: 151/508 (29.72%)
Target Accuracy: 117/2837 (4.12%)
Domain Accuracy: 778/3345 (23.26%)
Epoch 3/5 completed.
Epoch: 4/5



Testing: 100%|██████████| 32/32 [00:12<00:00,  2.66batch/s]

Test Results on DANN:
Source Accuracy: 239/508 (47.05%)
Target Accuracy: 154/2837 (5.43%)
Domain Accuracy: 532/3345 (15.90%)
Epoch 4/5 completed.
Epoch: 5/5



Testing: 100%|██████████| 32/32 [00:10<00:00,  2.96batch/s]


Test Results on DANN:
Source Accuracy: 364/508 (71.65%)
Target Accuracy: 248/2837 (8.74%)
Domain Accuracy: 518/3345 (15.49%)
Epoch 5/5 completed.


#### D -> W

In [ ]:
dslr_loader = load_data(data_path, 'dslr', 16, 'src')
webcam_loader = load_data(data_path, 'webcam', 16, 'tar')

print("Number of labels in Amazon dataset:", len(amazon_loader.dataset.classes))

# Create the model
encoder = Extractor().to(device)
classifier = Classifier(31).to(device)
discriminator = DomainClassifier().to(device)


# Train the model
train_dann(encoder, classifier, discriminator, dslr_loader, webcam_loader, 5, device)

Number of labels in Amazon dataset: 31
Training with the DANN adaptation method
Epoch: 1/5


Testing: 100%|██████████| 32/32 [00:12<00:00,  2.64batch/s]

Test Results on DANN:
Source Accuracy: 55/508 (10.83%)
Target Accuracy: 45/795 (5.66%)
Domain Accuracy: 751/1303 (57.64%)
Epoch 1/5 completed.
Epoch: 2/5



Testing: 100%|██████████| 32/32 [00:12<00:00,  2.64batch/s]

Test Results on DANN:
Source Accuracy: 81/508 (15.94%)
Target Accuracy: 63/795 (7.92%)
Domain Accuracy: 819/1303 (62.85%)
Epoch 2/5 completed.
Epoch: 3/5


...

Test Results on DANN:
Source Accuracy: 94/508 (18.50%)
Target Accuracy: 74/795 (9.31%)
Domain Accuracy: 803/1303 (61.63%)
Epoch 3/5 completed.
Epoch: 4/5



Testing: 100%|██████████| 32/32 [00:12<00:00,  2.66batch/s]

Test Results on DANN:
Source Accuracy: 165/508 (32.48%)
Target Accuracy: 117/795 (14.72%)
Domain Accuracy: 727/1303 (55.79%)
Epoch 4/5 completed.
Epoch: 5/5



Testing: 100%|██████████| 32/32 [00:13<00:00,  2.31batch/s]

Test Results on DANN:
Source Accuracy: 184/508 (36.22%)
Target Accuracy: 144/795 (18.11%)
Domain Accuracy: 634/1303 (48.66%)
Epoch 5/5 completed.


## MNIST Experiments

#### M -> U

In [ ]:
torch.cuda.empty_cache()
# Create the model
encoder = Extractor().to(device)
classifier = Classifier(10).to(device)
discriminator = DomainClassifier().to(device)


# Train the model
train_dann(encoder, classifier, discriminator, mnist_train_subset_loader, usps_train_subset_loader, 5, device, mnist_test_subset_loader, usps_test_subset_loader)

Training with the DANN adaptation method
Epoch: 1/5


Testing: 100%|██████████| 125/125 [00:15<00:00,  7.82batch/s]


Test Results on DANN:
Source Accuracy: 974/5000 (19.48%)
Target Accuracy: 913/1000 (91.30%)
Domain Accuracy: 839/6000 (13.98%)
Epoch 1/5 completed.
Epoch: 2/5


Testing: 100%|██████████| 125/125 [00:15<00:00,  7.91batch/s]


Test Results on DANN:
Source Accuracy: 991/5000 (19.82%)
Target Accuracy: 943/1000 (94.30%)
Domain Accuracy: 1125/6000 (18.75%)
Epoch 2/5 completed.
Epoch: 3/5


...

Test Results on DANN:
Source Accuracy: 992/5000 (19.84%)
Target Accuracy: 937/1000 (93.70%)
Domain Accuracy: 1006/6000 (16.77%)
Epoch 3/5 completed.
Epoch: 4/5


Testing: 100%|██████████| 125/125 [00:15<00:00,  7.96batch/s]


Test Results on DANN:
Source Accuracy: 992/5000 (19.84%)
Target Accuracy: 954/1000 (95.40%)
Domain Accuracy: 1161/6000 (19.35%)
Epoch 4/5 completed.
Epoch: 5/5


Testing: 100%|██████████| 125/125 [00:15<00:00,  7.99batch/s]

Test Results on DANN:
Source Accuracy: 993/5000 (19.86%)
Target Accuracy: 932/1000 (93.20%)
Domain Accuracy: 1073/6000 (17.88%)
Epoch 5/5 completed.


#### U -> M

In [ ]:
torch.cuda.empty_cache()
# Create the model
encoder = Extractor().to(device)
classifier = Classifier(10).to(device)
discriminator = DomainClassifier().to(device)


# Train the model
train_dann(encoder, classifier, discriminator, usps_train_subset_loader, mnist_train_subset_loader, 5, device, usps_test_subset_loader, mnist_test_subset_loader)

Training with the DANN adaptation method
Epoch: 1/5


Testing: 100%|██████████| 125/125 [00:15<00:00,  7.99batch/s]


Test Results on DANN:
Source Accuracy: 936/1000 (93.60%)
Target Accuracy: 859/5000 (17.18%)
Domain Accuracy: 1094/6000 (18.23%)
Epoch 1/5 completed.
Epoch: 2/5


Testing: 100%|██████████| 125/125 [00:15<00:00,  7.99batch/s]


Test Results on DANN:
Source Accuracy: 951/1000 (95.10%)
Target Accuracy: 901/5000 (18.02%)
Domain Accuracy: 1172/6000 (19.53%)
Epoch 2/5 completed.
Epoch: 3/5


...

Test Results on DANN:
Source Accuracy: 962/1000 (96.20%)
Target Accuracy: 954/5000 (19.08%)
Domain Accuracy: 1155/6000 (19.25%)
Epoch 3/5 completed.
Epoch: 4/5


Testing: 100%|██████████| 125/125 [00:15<00:00,  7.98batch/s]


Test Results on DANN:
Source Accuracy: 967/1000 (96.70%)
Target Accuracy: 956/5000 (19.12%)
Domain Accuracy: 1083/6000 (18.05%)
Epoch 4/5 completed.
Epoch: 5/5


Testing: 100%|██████████| 125/125 [00:15<00:00,  7.91batch/s]

Test Results on DANN:
Source Accuracy: 969/1000 (96.90%)
Target Accuracy: 950/5000 (19.00%)
Domain Accuracy: 1147/6000 (19.12%)
Epoch 5/5 completed.


#### S -> M

In [ ]:
torch.cuda.empty_cache()
# Create the model
encoder = Extractor().to(device)
classifier = Classifier(10).to(device)
discriminator = DomainClassifier().to(device)


# Train the model
train_dann(encoder, classifier, discriminator, svhn_train_subset_loader, mnist_train_subset_loader, 5, device, svhn_test_subset_loader, mnist_test_subset_loader)
#

Training with the DANN adaptation method
Epoch: 1/5


Testing: 100%|██████████| 625/625 [01:17<00:00,  8.03batch/s]


Test Results on DANN:
Source Accuracy: 4020/5000 (80.40%)
Target Accuracy: 3198/5000 (63.96%)
Domain Accuracy: 4683/10000 (46.83%)
Epoch 1/5 completed.
Epoch: 2/5


Testing: 100%|██████████| 625/625 [01:18<00:00,  8.00batch/s]


Test Results on DANN:
Source Accuracy: 4498/5000 (89.96%)
Target Accuracy: 3595/5000 (71.90%)
Domain Accuracy: 6868/10000 (68.68%)
Epoch 2/5 completed.
Epoch: 3/5


...

Test Results on DANN:
Source Accuracy: 4511/5000 (90.22%)
Target Accuracy: 3719/5000 (74.38%)
Domain Accuracy: 5182/10000 (51.82%)
Epoch 3/5 completed.
Epoch: 4/5


Testing: 100%|██████████| 625/625 [01:18<00:00,  7.92batch/s]


Test Results on DANN:
Source Accuracy: 4631/5000 (92.62%)
Target Accuracy: 3889/5000 (77.78%)
Domain Accuracy: 4824/10000 (48.24%)
Epoch 4/5 completed.
Epoch: 5/5


Testing: 100%|██████████| 625/625 [01:18<00:00,  7.92batch/s]

Test Results on DANN:
Source Accuracy: 4636/5000 (92.72%)
Target Accuracy: 3755/5000 (75.10%)
Domain Accuracy: 4955/10000 (49.55%)
Epoch 5/5 completed.
